In [ ]:
import pandas as pd
import numpy as np

In [ ]:
raw_leads = pd.read_csv("../Data/Raw/leads.csv")
raw_offers = pd.read_csv("../Data/Raw/offers.csv")

In [ ]:
# Converting string "nan" into null values sice they generated False Negatives values 
raw_leads[raw_leads["Id"] == "nan"] = np.nan
raw_offers[raw_offers["Id"] == "nan"] = np.nan

# Drop duplicated IDs
raw_offers_no_duplicates = raw_offers.copy()
raw_offers_no_duplicates["Id"] = raw_offers["Id"].drop_duplicates()

# Creating a dataframe with out null values on ID column
leads_no_ID_nan = raw_leads[raw_leads["Id"].isna() == False]
offers_no_ID_nan = raw_offers_no_duplicates[raw_offers_no_duplicates["Id"].isna() == False]

# Filling the null nada with a new category (Not_specified) 
leads_no_ID_nan.fillna("Not_Specified", inplace=True)
offers_no_ID_nan.fillna("Not_Specified", inplace=True)

In [ ]:
leads_df = leads_no_ID_nan
offers_df = offers_no_ID_nan

In [ ]:
# Changing the type of variable of dates fields
offers_df["Created Date"] = pd.to_datetime(offers_df["Created Date"])
offers_df["Close Date"] = pd.to_datetime(offers_df["Close Date"])

In [ ]:
# Create a new column to know how many days took the sales process
offers_df["Days"] = offers_df["Close Date"] - offers_df["Created Date"]

# Creating a column to know if there was applied a discount or not
offers_df["Discount"] = offers_df["Discount code"] == "Not_Specified"

In [ ]:
# Transforms the converted variable to bool
leads_df.Converted = leads_df.Converted.astype('bool')

# Removing some unnecessary columns for the algorythm like name or reason why was discarded 
# Discarded reason is a value that is given after know that the person was not converted it wont be an input for this model
leads_reduced = leads_df.drop(["First Name", "Discarded/Nurturing Reason"], axis=1)